In [1]:
import os
# CUDAVISIBLE DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import transformers

/root/miniconda3/envs/snapkv_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from snapkv.monkeypatch.monkeypatch import replace_llama, replace_mistral, replace_mixtral

In [3]:
from importlib.metadata import version

def check_version():
    try:
        transformers_version = version("transformers")
    except Exception as e:
        print(f"Transformers not installed: {e}")
    return transformers_version

check_version()

'4.37.0'

In [4]:
replace_mistral()

In [5]:
from fastchat.model import load_model, get_conversation_template

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "/root/autodl-tmp/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    use_flash_attention_2=True
    )

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


In [10]:
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/Mistral-7B-Instruct-v0.2",
                                          use_fast=False)

In [11]:
with open('snapkv.txt', 'r') as f:
    content = f.read().strip()

In [12]:
question = "\n What is the repository of SnapKV?"

In [13]:
conv = get_conversation_template("/root/autodl-tmp/Mistral-7B-Instruct-v0.2")
conv.messages = []
conv.append_message(conv.roles[0],content + question)
# conv.append_message(conv.roles[0],"Who is Kobe Bryant?")
conv.append_message(conv.roles[1], None)

In [14]:
prompt = conv.get_prompt()

In [15]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [16]:
input_ids_len = input_ids.size(1)
print(input_ids_len)

17395


In [17]:
outputs = model.generate(input_ids.cuda(), max_new_tokens=200, do_sample=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
print(tokenizer.decode(outputs[0][input_ids_len:], skip_special_tokens=True))

The repository of SnapKV is available at <https://github.com/FasterDecoding/SnapKV>.
